# Avichai Chen - Assigment 3

### Import of different libraries for the benefit of the assignment:

In [1]:
import pandas as pd
import json
import requests
import urllib.request

### Read the txt file into list:

In [2]:
f = open('dests.txt',"r")
dest = f.read().split()

### Function to read from Googleapis all parameters:

In [3]:
def Googleapis_data (dest,key):
    serviceurl='https://maps.googleapis.com/maps/api/distancematrix/json?'
    d = {}
    d['destinations'] = dest
    d['origins'] = 'Tel Aviv'
    d['key'] = key
    url = serviceurl + urllib.parse.urlencode(d)

    serviceurl2="https://maps.googleapis.com/maps/api/geocode/json?"
    d2 = {}
    d2['address'] = dest
    d2['key'] = key
    url2 = serviceurl2 + urllib.parse.urlencode(d2)
    
    try:
        response = requests.get(url)
        response2 = requests.get(url2)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        elif not response2.status_code == 200:
            print("HTTP error",response2.status_code)
            
        else:
            try:
                response_data = response.json()
                distance = response_data['rows'][0]['elements'][0]['distance']['value'] / 1000
                seconds = response_data['rows'][0]['elements'][0]['duration']['value']
                minutes = (seconds // 60) % 60
                hours = seconds // 3600
                duration = str(hours) + " hours + " + str (minutes) + " minutes"  
                
                response_data2 = response2.json()
                longitude = (response_data2['results'][0]['geometry']['location']['lng'])
                latitude = (response_data2['results'][0]['geometry']['location']['lat'])
                
                return {"Target":dest,"Distance_km":distance,"Duration(hour+minutes)":duration,"Longitude":longitude,"Latitude":latitude}
            
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")

### Calculate the parametes with the function for each target and enter to data frame:

In [4]:
df = pd.DataFrame()
# key = "enter your key"

for d in dest:
    new_row = pd.DataFrame(Googleapis_data(d,key), index=[0])
    df = pd.concat([df, new_row], ignore_index=True)

### The information should be stored in a dataframe of the following structure:

In [5]:
df

,Target,Distance_km,Duration(hour+minutes),Longitude,Latitude
0,Istanbul,1815.227,21 hours + 2 minutes,28.978359,41.008238
1,Amsterdam,4532.541,48 hours + 3 minutes,4.904139,52.367573
2,Valletta,3792.883,50 hours + 50 minutes,14.514100,35.899237
3,Basel,4092.805,44 hours + 2 minutes,7.588576,47.559599
4,Doha,2164.477,22 hours + 38 minutes,51.531040,25.285447


### Find the 3 cities furthest from Tel Aviv:

In [6]:
df2 = df.sort_values('Distance_km',ascending=False)[0:3]
df2['Target']

1    Amsterdam
3        Basel
2     Valletta
Name: Target, dtype: object